# Sentiment Classification on IMDB Movie Reviews

In this notebook we will build **three simple models** to classify IMDB movie reviews as positive or negative:

- Simple RNN
- 1D CNN
- LSTM

We will:
1. Load the IMDB dataset from Keras.
2. Preprocess the text (token IDs + padding).
3. Build and train the three models.
4. Compare accuracy and training time.
5. Write a short conclusion.

## 1. Imports

In [13]:
import time

import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam

## 2. Load the IMDB dataset

The IMDB dataset is already **tokenized as integers**. Each review is a list of word indices.
We will limit the vocabulary size so the models are small and easy to train.

In [14]:
# Number of words to keep (most frequent)
vocab_size = 10000

# Load data (already tokenized as integer sequences)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

print("Number of training samples:", len(x_train))
print("Number of test samples:", len(x_test))
print("Example review (first 20 word indices):", x_train[0][:20])
print("Label (0 = negative, 1 = positive):", y_train[0])

Number of training samples: 25000
Number of test samples: 25000
Example review (first 20 word indices): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25]
Label (0 = negative, 1 = positive): 1


## 3. Preprocess text (pad sequences)

Reviews have **different lengths**. Neural networks work better if all sequences have the same length.
So we will pad (or cut) each review to a fixed length.

In [15]:
# Maximum length of each review (in number of word indices)
maxlen = 200  # small to make models faster

# Pad sequences with zeros at the beginning if they are shorter than maxlen
x_train_padded = pad_sequences(x_train, maxlen=maxlen)
x_test_padded = pad_sequences(x_test, maxlen=maxlen)

print("Shape of x_train_padded:", x_train_padded.shape)
print("Shape of x_test_padded:", x_test_padded.shape)

Shape of x_train_padded: (25000, 200)
Shape of x_test_padded: (25000, 200)


We will use the **same embedding layer settings** for all three models so that the comparison is fair.

In [16]:
embedding_dim = 32  # size of word vectors
batch_size = 128
epochs = 3  # keep small so training is quick

## 4. Simple RNN model

A **Simple RNN** reads one word at a time and keeps a small memory of previous words.

In [17]:
# Build Simple RNN model
rnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')  # binary classification
])

rnn_model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])



In [18]:
# Train Simple RNN
start_time = time.time()
history_rnn = rnn_model.fit(
    x_train_padded,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=1
)
rnn_time = time.time() - start_time

# Evaluate on test data
rnn_loss, rnn_acc = rnn_model.evaluate(x_test_padded, y_test, verbose=0)
print(f"RNN test accuracy: {rnn_acc:.4f}")
print(f"RNN training time: {rnn_time:.2f} seconds")

rnn_model.summary()

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.7447 - loss: 0.5067 - val_accuracy: 0.8168 - val_loss: 0.4036
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.8820 - loss: 0.2935 - val_accuracy: 0.8604 - val_loss: 0.3315
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.9164 - loss: 0.2210 - val_accuracy: 0.8696 - val_loss: 0.3399
RNN test accuracy: 0.8594
RNN training time: 15.14 seconds


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 200, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 966,341 (3.69 MB)

 Trainable params: 322,113 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 644,228 (2.46 MB)

## 5. 1D CNN model

A **1D CNN** applies convolution filters over neighboring words. It is good at
finding local patterns (like short phrases). It can also train faster because
it can process many words in parallel.

In [19]:
# Build 1D CNN model
cnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])



In [20]:
# Train 1D CNN
start_time = time.time()
history_cnn = cnn_model.fit(
    x_train_padded,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=1
)
cnn_time = time.time() - start_time

# Evaluate on test data
cnn_loss, cnn_acc = cnn_model.evaluate(x_test_padded, y_test, verbose=0)
print(f"CNN test accuracy: {cnn_acc:.4f}")
print(f"CNN training time: {cnn_time:.2f} seconds")


cnn_model.summary()

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7174 - loss: 0.6003 - val_accuracy: 0.7976 - val_loss: 0.4745
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8266 - loss: 0.3995 - val_accuracy: 0.8438 - val_loss: 0.3680
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8763 - loss: 0.3015 - val_accuracy: 0.8648 - val_loss: 0.3259
CNN test accuracy: 0.8577
CNN training time: 6.89 seconds


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 200, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 198, 32)        │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 32)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 969,413 (3.70 MB)

 Trainable params: 323,137 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 646,276 (2.47 MB)

## 6. LSTM model

An **LSTM (Long Short-Term Memory)** network is a special type of RNN that is
better at remembering long-term information and avoiding the vanishing gradient problem.

In [21]:
# Build LSTM model
lstm_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(optimizer=Adam(learning_rate=0.001),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [22]:
# Train LSTM
start_time = time.time()
history_lstm = lstm_model.fit(
    x_train_padded,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=1
)
lstm_time = time.time() - start_time

# Evaluate on test data
lstm_loss, lstm_acc = lstm_model.evaluate(x_test_padded, y_test, verbose=0)
print(f"LSTM test accuracy: {lstm_acc:.4f}")
print(f"LSTM training time: {lstm_time:.2f} seconds")


lstm_model.summary()

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 71ms/step - accuracy: 0.7501 - loss: 0.5021 - val_accuracy: 0.8380 - val_loss: 0.3705
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 11s 68ms/step - accuracy: 0.8941 - loss: 0.2708 - val_accuracy: 0.8614 - val_loss: 0.3356
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 11s 69ms/step - accuracy: 0.9253 - loss: 0.2047 - val_accuracy: 0.8696 - val_loss: 0.3190
LSTM test accuracy: 0.8659
LSTM training time: 35.04 seconds


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 200, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 985,061 (3.76 MB)

 Trainable params: 328,353 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 656,708 (2.51 MB)

## 7. Compare accuracy and training time

In [23]:
print("\n=== Test Accuracy ===")
print(f"Simple RNN: {rnn_acc:.4f}")
print(f"1D CNN    : {cnn_acc:.4f}")
print(f"LSTM      : {lstm_acc:.4f}")

print("\n=== Training Time (seconds) ===")
print(f"Simple RNN: {rnn_time:.2f}")
print(f"1D CNN    : {cnn_time:.2f}")
print(f"LSTM      : {lstm_time:.2f}")


=== Test Accuracy ===
Simple RNN: 0.8594
1D CNN    : 0.8577
LSTM      : 0.8659

=== Training Time (seconds) ===
Simple RNN: 15.14
1D CNN    : 6.89
LSTM      : 35.04


## 8. Short conclusion 


- **Which model performs best and why?**  
  Usually, the **LSTM** gets the best accuracy because it remembers long-term dependencies in the text better than a simple RNN.

- **Which model was fastest?**  
  Often, the **1D CNN** trains fastest because convolutions can be parallelized and the model is shallow.

- **Which performed best in your run?**  
  LSTM gave the highest accuracy but the 1D CNN worked besT in terms of computational speed.

- **Why might LSTM outperform a Simple RNN?**  
  LSTMs have a special internal structure (gates and cell state) that helps them **remember important information for longer** and **forget unimportant information**. This makes them better at understanding the meaning of long sentences.

- **Why might CNN train faster?**  
  CNNs look at several words at once using filters and do not have to process the sequence step-by-step like RNNs. This parallel processing makes training faster while still capturing useful local patterns (like important phrases).